In [1]:
import pandas as pd
import functions as fn

In [ ]:
### TODO
### process discharge summaries to only include text sections
### iterate through entire dataframe, extract all discharge summaries, save as text file

Questions:
- Should this be a NER task?
- If it is a NER task - would it be: can I identify which symptoms are potentially related to irAEs?
- Should there be any additional considerations? e.g. which files belong to people with chemo, etc
- removing:
    - everything above HOPC
    - everything below Discharge Instructions

In [2]:
file_path = "./data/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/note/"

In [3]:
df = pd.read_csv(file_path+"discharge.csv")

In [6]:
test = df["text"][400]

In [7]:
with open('medical_report_2.txt', 'w') as f:
    f.write(test)

In [42]:
type(test)

str

In [7]:
dir = "./data"
label_list = [
   "Inflammatory symptom/sign",
   "Inflammatory investigation results",
   "Cancer",
   "Immunotherapy",
   "Chemotherapy" 
]
database = "irAE"
model = "en_core_web_sm"
annotate = fn.AnnotationProcess(dir, label_list, database, model)
test_text = annotate.card_generator(test)

with open('test.txt', 'w') as f:
    f.write(test_text)

annotate.database_connect()

annotate.prodigy_connect("ner.manual", "./test.txt")


Using 4 label(s): Medication, Frequency, Dose, Route

✨  Starting the web server at http://localhost:8080 ...
Open the app in your browser and start annotating!



In [25]:
t = "this is a sentence. \n this is another sentence"
sentences = t.replace("\n", "")
sentences = sentences.split(". ")
sentences

['this is a sentence', ' this is another sentence']